## Benemérita Universidad Autónoma de Puebla, México.
### Julia V1.5.0
#### Dr. Enrique Ricardo Pablo Buendia Lozada

Estadística para la validación de Pruebas en:
- Cultura Física, 
- Educación Física,
- Fisioterapia, 
- Recreación, 
- Ciencias de la actividad Física, 
- Deportes, 
- entre otras.

##### Versión: V1 Julia 1.5.0
##### Caso: Paramétrico
##### Febrero 2021

In [1]:
using Gadfly, Cairo

In [2]:
using Statistics, Distributions

In [3]:
using CSV, DataFrames

In [4]:
using SimpleANOVA

In [5]:
df=CSV.read("datos.csv";header=false)      # Ejemplo

,Column1,Column2,Column3
,Int64,Float64,Float64
1,1,1.1,0.89
2,2,1.9,2.0001
3,3,2.9,3.001
4,4,3.9,4.001
5,5,4.9,5.0001
6,6,5.9,6.001
7,7,7.0,7.0
8,6,6.0,5.8999
9,5,5.0,4.899


In [6]:
#names(df)
#rename!(df,["c1","c2","c3"])              # se puede omitir, pues no importan los títulos

In [7]:
## Bland - Altman

function bland_altman(x,y,i,j)
    
    n1=length(x)
    n2=length(y)
    if n1==n2 
        mn=0.5*(x+y)
        dd=x-y
        md=sum(dd)./length(dd)
       
        vd=var(dd)
        
        ls=md+2*(vd)^(0.5)
        li=md-2*(vd)^(0.5)
        
        # View Blant - Altman
        p=plot(x=mn,y=dd,Geom.point,yintercept=[ls, li],Geom.hline(color=["red","red"]),Guide.title("Bland-Altman"))
        img = SVG("img_muestras_$i-$j.svg", 6inch, 4inch)
        draw(img, p)
        draw(PNG("img_muestras_$i-$j.png", 6inch, 4inch), p)
        
    else
        println("Las muestras no son del mismo tamaño...")
        
    end
end

bland_altman (generic function with 1 method)

In [8]:
function ver_outliers(x,v1)    
    outl=0
    lsbox1=quantile(x,0.75)+1.5(quantile(x,.75)-quantile(x,.25))
    libox1=quantile(x,0.25)-1.5(quantile(x,.75)-quantile(x,.25))
    # a[a .> 4]
    os=x[x .> lsbox1]
    if length(os)>=1
        println("Outliers: ",os," en muestra $v1 ")
        outl=1
    end
    os=[]
    oi=x[x .< libox1]
    if length(oi)>=1
        println("Outliers: ",oi," en muestra $v1 ")
        outl=1
    end
    oi=[]
    if outl==0
        println("Sin outliers ...muestra $v1 ")
    end
end

ver_outliers (generic function with 1 method)

In [9]:
# coeficiente de de correlación de concordancia de Lawrence - Lin
#
function ll(x,y)
    rc=(2*cov(x,y))/(var(x)+var(y)+(mean(x)-mean(y))^2)
    print("Lawrence - Lin $rc ")
    
    if rc >= 0. && rc < 0.01
        println("pobre --> Interpretación de Landis y Koch (1977)")
    elseif rc >=0.01 && rc<=0.20
        println("leve  --> Interpretación de Landis y Koch (1977)")
    elseif rc>=0.21 && rc <=0.40
        println("regular  --> Interpretación de Landis y Koch (1977)")
    elseif rc>=0.41 && rc<=0.60
        println("moderado  --> Interpretación de Landis y Koch (1977)")
    elseif rc>=0.61 && rc<=0.80
        println("substancial  --> Interpretación de Landis y Koch (1977)")
    elseif rc>=0.81 && rc<=1
        println("casi perfecto  --> Interpretación de Landis y Koch (1977)")
    end
end

ll (generic function with 1 method)

In [25]:
# realizar todas las combinaciones de 2
# de todas las muestras disponibles

muest   =ncol(df)
muestm1 = muest-1
i=1

while i<=muestm1
    j=i+1
    while j<=muest
       x = df[i]
       y = df[j]
       
        ver_outliers(x,i)
        ver_outliers(y,j)
        co=cor(x,y) # coeficiente de correlación de PEARSON
        print("Pearson: $co ")
        if co >= 0. && co < 0.01
            println("pobre --> Interpretación de Landis y Koch (1977)")
        elseif co >=0.01 && co<=0.20
            println("leve  --> Interpretación de Landis y Koch (1977)")
        elseif co >=0.21 && co <=0.40
            println("regular  --> Interpretación de Landis y Koch (1977)")
        elseif co >=0.41 && co <=0.60
            println("moderado  --> Interpretación de Landis y Koch (1977)")
        elseif co >=0.61 && co <=0.80
            println("substancial  --> Interpretación de Landis y Koch (1977)")
        elseif co >=0.81 && co <=1
            println("casi perfecto  --> Interpretación de Landis y Koch (1977)")
        end
        
        bland_altman(x,y,i,j) # guarda las gráficas para verlas en el navegador
        ll(x,y)   # coeficiente de correlación de  concordancia de Lawrence - Lin
        println(" imagen de muestras $i $j guardada ...")
        
        j=j+1
    end
    i=i+1
end 



Sin outliers ...muestra 1 
Sin outliers ...muestra 2 
Pearson: 0.9993408977454561 casi perfecto  --> Interpretación de Landis y Koch (1977)
Lawrence - Lin 0.999244478709616 casi perfecto  --> Interpretación de Landis y Koch (1977)
 imagen de muestras 1 2 guardada ...
Sin outliers ...muestra 1 
Sin outliers ...muestra 3 
Pearson: 0.9992711791540291 casi perfecto  --> Interpretación de Landis y Koch (1977)
Lawrence - Lin 0.9987109546635097 casi perfecto  --> Interpretación de Landis y Koch (1977)
 imagen de muestras 1 3 guardada ...
Sin outliers ...muestra 2 
Sin outliers ...muestra 3 
Pearson: 0.997798524922192 casi perfecto  --> Interpretación de Landis y Koch (1977)
Lawrence - Lin 0.9974396413115486 casi perfecto  --> Interpretación de Landis y Koch (1977)
 imagen de muestras 2 3 guardada ...


In [31]:
#tabla de valoración
function tablaval(x)
    dest=(var(x))^(0.5)
    media=mean(x)
    dest1=media+dest
    dest2=media+dest*2
    dest3=media+dest*3
    ndest1=media-dest
    ndest2=media-dest*2
    ndest3=media-dest*3
     ff=x[x .> dest3] 
    f=length(ff)
    println(" $dest3 - o más     $f")
     ff=x[(x .> dest2) .& (x .<= dest3)] 
    f=length(ff)
    println(" $dest2 - $dest3     $f")
     ff=x[(x .> dest1) .& (x .<= dest2)] 
    f=length(ff)
    println(" $dest1 - $dest2     $f")
     ff=x[(x .> media) .& (x .<= dest1)] 
    f=length(ff)
    println(" $media - $dest1     $f")
    
     ff=x[(x .> ndest1) .& (x .<= media)]
    f=length(ff)
    println(" $ndest1 - $media     $f")
     ff=x[(x .> ndest2) .& (x .<= ndest1)] 
    f=length(ff)
    println(" $ndest2 - $ndest1     $f")
     ff=x[(x .> ndest3) .& (x .<= ndest2)]
    f=length(ff)
    println(" $ndest3 - $ndest2     $f")
     ff=x[x .<= ndest3]
    f=length(ff)
    println(" o menos - $ndest3     $f")
end

tablaval (generic function with 1 method)

In [27]:
function normalk2(x,i)
# D'Agostino-Pearson's K2 test for assessing normality of data using skewness and kurtosis.
# Adaptado y traducido de http://welles.dm.unibo.it/~simoncin/DagosPtest.m
alpha = 0.05
n = length(x)
s1=sum(x)
s2 = sum(x.^2)
s3 = sum(x.^3)
s4 = sum(x.^4)
ss = s2-(s1^2/n)
v = ss/(n-1)
k3 = ((n*s3)-(3*s1*s2)+((2*(s1^3))/n))/((n-1)*(n-2))
g1 = k3/sqrt(v^3)
k4 = ((n+1)*((n*s4)-(4*s1*s3)+(6*(s1^2)*(s2/n))-((3*(s1^4))/(n^2)))/((n-1)*(n-2)*(n-3)))-((3*(ss^2))/((n-2)*(n-3)))
g2 = k4/v^2
eg1 = ((n-2)*g1)/sqrt(n*(n-1))  #measure of skewness
eg2 = ((n-2)*(n-3)*g2)/((n+1)*(n-1))+((3*(n-1))/(n+1))  #measure of kurtosis
A = eg1*sqrt(((n+1)*(n+3))/(6*(n-2)))
B = (3*((n^2)+(27*n)-70)*((n+1)*(n+3)))/((n-2)*(n+5)*(n+7)*(n+9))
C = sqrt(2*(B-1))-1
D = sqrt(C)
E = 1/sqrt(log(D))
F = A/sqrt(2/(C-1))
Zg1 = E*log(F+sqrt(F^2+1))

G = (24*n*(n-2)*(n-3))/((n+1)^2*(n+3)*(n+5))
H = ((n-2)*(n-3)*abs(g2))/((n+1)*(n-1)*sqrt(G))
J = ((6*(n^2-(5*n)+2))/((n+7)*(n+9)))*sqrt((6*(n+3)*(n+5))/((n*(n-2)*(n-3))))
K = 6+((8/J)*((2/J)+sqrt(1+(4/J^2))))
L = (1-(2/K))/(1+H*sqrt(2/(K-4)))
Zg2 = (1-(2/(9*K))-L^(1/3))/sqrt(2/(9*K))

K2 = Zg1^2 + Zg2^2  #D'Agostino-Pearson statistic
X2 = K2  #approximation to chi-distribution

df = 2.  #degrees of freedom
P=1-ccdf(Chisq(df), X2)
#println("P = $P ")
if P>alpha
    println("La muestra $i se distribuye normalmente ...")
    println("Tabla de Valoración    [NORMA]")
    println("Intervalos                            frecuencia")
    tablaval(x)
        println()
        println()
else
    println("La muestra $i NO viene de una distribución normal")
end
end

normalk2 (generic function with 1 method)

In [28]:
# comprobar si todas las muestras provienen de la distribución normal
ccc=ncol(df)
i=1

nren=length(df[1])
while i<=ccc
     x = df[i]
     normalk2(x,i)
     i=i+1
end


La muestra 1 se distribuye normalmente ...
Tabla de Valoración    [NORMA]
Intervalos                            frecuencia
 9.662512472218438 - o más     0
 7.698085237889215 - 9.662512472218438     0
 5.733658003559992 - 7.698085237889215     3
 3.769230769230769 - 5.733658003559992     4
 1.8048035349015463 - 3.769230769230769     4
 -0.1596236994276765 - 1.8048035349015463     2
 -2.124050933756899 - -0.1596236994276765     0
 o menos - -2.124050933756899     0


La muestra 2 se distribuye normalmente ...
Tabla de Valoración    [NORMA]
Intervalos                            frecuencia
 9.596301773277634 - o más     0
 7.6462524642363725 - 9.596301773277634     0
 5.696203155195109 - 7.6462524642363725     3
 3.746153846153846 - 5.696203155195109     4
 1.796104537112583 - 3.746153846153846     4
 -0.15394477192868017 - 1.796104537112583     2
 -2.103994080969943 - -0.15394477192868017     0
 o menos - -2.103994080969943     0


La muestra 3 se distribuye normalmente ...
Tabla de Valo

In [29]:
mat1=convert(Matrix,df)
# verificar igualdad de varianzas [Homocedasticidad]
l=levene(mat1)
println("Si F > p rechazar Ho: Las muestras tienen varianzas iguales ...")
# consultar : https://github.com/BioTurboNick/SimpleANOVA.jl
println(l)

Si F > p rechazar Ho: Las muestras tienen varianzas iguales ...

Analysis of Variance Results

Effect           SS  DF          MS           F         p          ω²
---------------------------------------------------------------------
 Total  34.579       38                                              
Groups   0.00485383   2  0.00242692  0.00252701  0.997476  -0.0539101
 Error  34.5741      36  0.960392                                    



In [30]:
# verificar igualdad de promedios  [ANOVA de una vía]
p=anova(mat1)
println("Si F > p rechazar Ho: Las muestras tienen medias iguales ...")
# consultar : https://github.com/BioTurboNick/SimpleANOVA.jl
println(p)

Si F > p rechazar Ho: Las muestras tienen medias iguales ...

Analysis of Variance Results

Effect           SS  DF         MS           F         p          ω²
--------------------------------------------------------------------
 Total  139.229      38                                             
     A    0.0264066   2  0.0132033  0.00341459  0.996592  -0.0538595
 Error  139.202      36  3.86673                                    

